# "Equivalent" Heterogeneous System

I will consider a pin-cell 5cm wide with a 1cm diameter pin in the center.  I will use Sodium at a nominal density of 0.97 g/cc to surround the pin and the pin will consist of "Plutonium-239" with a density of 0.925 g/cc.  Obviously the Pu239 density is bogus but that's what it needs to be set at in order to get the overall weight fraction the same as the homogeneous case.  Effectively we will be ignoring all other atoms in a realistic system besides Pu-239 and Na.

This is fake but at least gives us a model of the effect of "Heterogeneous-izing" the system

In [1]:
%matplotlib inline

import openmc
import numpy as np
import matplotlib.pyplot as plt


In [4]:
Na = openmc.Material(name='Na');
Na.add_nuclide('Na23',1);
Na.set_density('g/cc',0.97);

Pu = openmc.Material(name='Pu');
Pu.add_nuclide('Pu239',1);
Pu.set_density('g/cc',0.925);

materials = openmc.Materials([Na,Pu]);
materials.export_to_xml();

H = 400; # cm, height of the core
P = 5; # cm, pitch of the cell
pin_cell = openmc.model.RectangularParallelepiped(-P/2.,P/2.,-P/2.,P/2.,
                                             -H/2.,H/2.,
                                             boundary_type='reflective');

H_pin = 399;
r_pin = 0.5
pin = openmc.model.RightCircularCylinder([0.,0.,-H_pin/2.],
                                        H_pin,r_pin);

pin_c = openmc.Cell();
pin_c.region = -pin;
pin_c.fill = Pu;
pin_c.temperature = 600;

cool_c = openmc.Cell();
cool_c.region = +pin & -pin_cell;
cool_c.fill = Na;
cool_c.temperature = 600;

root_u = openmc.Universe();
root_u.add_cells([pin_c,cool_c]);

geometry = openmc.Geometry(root_u);
geometry.export_to_xml();





nu_tally = openmc.Tally(name='nu');
nu_tally.scores = ['nu-fission']


abs_tally = openmc.Tally(name='abs');
abs_tally.scores = ['absorption']
abs_tally.nuclides = ['Pu239','Na23'];

fiss_tally = openmc.Tally(name='fission');
fiss_tally.scores = ['fission'];

capture_tally = openmc.Tally(name='n_gamma');
capture_tally.scores = ['(n,gamma)'];
capture_tally.nuclides = ['Pu239'];

flux_tally = openmc.Tally(name='flux');
flux_tally.scores = ['flux'];
N = 1001
energy_bins = np.logspace(-3,7,num=N);
energy_filter = openmc.EnergyFilter(values=energy_bins);

flux_tally.filters = [energy_filter];

tallies = openmc.Tallies([nu_tally, abs_tally, fiss_tally,
                         capture_tally, flux_tally]);
tallies.export_to_xml();

settings = openmc.Settings();
settings.batches = 300;
settings.inactive = 100;
settings.particles = 20000;

R = r_pin;

bounds = [-R,-R,-H/2.,R,R,H/2.];
uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                   only_fissionable=True);
settings.source = openmc.source.Source(space=uniform_dist);

settings.temperature['method']='interpolation';

settings.export_to_xml();

In [5]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

      178/1    1.55947    1.55886 +/- 0.00126
      179/1    1.55207    1.55877 +/- 0.00125
      180/1    1.55654    1.55875 +/- 0.00123
      181/1    1.54974    1.55863 +/- 0.00122
      182/1    1.56782    1.55875 +/- 0.00121
      183/1    1.57503    1.55894 +/- 0.00121
      184/1    1.54245    1.55875 +/- 0.00121
      185/1    1.55466    1.55870 +/- 0.00120
      186/1    1.56980    1.55883 +/- 0.00119
      187/1    1.55449    1.55878 +/- 0.00118
      188/1    1.54015    1.55857 +/- 0.00119
      189/1    1.56540    1.55864 +/- 0.00118
      190/1    1.55644    1.55862 +/- 0.00116
      191/1    1.54932    1.55852 +/- 0.00115
      192/1    1.55372    1.55846 +/- 0.00114
      193/1    1.57496    1.55864 +/- 0.00114
      194/1    1.54682    1.55851 +/- 0.00114
      195/1    1.55808    1.55851 +/- 0.00113
      196/1    1.56357    1.55856 +/- 0.00112
      197/1    1.54485    1.55842 +/- 0.00111
      198/1    1.54921    1.55833 +/- 0.00111
      199/1    1.57194    1.55847 

I have to admit that I'm pretty freaked out as to how much the $k_{\infty}